In [2]:
import finlab
from finlab import data
import pandas as pd
import datetime as dt
import numpy as np
import math
from dotenv import load_dotenv
import os
import time
from finlab.backtest import sim
import scipy.stats as stats

def config():
    load_dotenv()


    
config()

finlab.login(str(os.getenv('FINLAB_API')))

輸入成功!


In [65]:
data.set_universe(market='TSE')
市值 = data.get('etl:market_value')

# 40-42 nan each company(using date as the minimal unit of time)
自由現金流 = data.get('fundamental_features:自由現金流量').index_str_to_date()

#15 -16 nan each company(using date as the minimal unit of time)
股東權益報酬率 = data.get('fundamental_features:ROE稅後').index_str_to_date()


# 131-141 nan each company(using date as the minimal unit of time)
營業利益成長率 = data.get('fundamental_features:營業利益成長率').index_str_to_date()


營業收入淨額 = data.get('financial_statement:營業收入淨額')


# 19-33 Nan in each company (using date as the minimal unit of time)
市值營收比 = 市值 / 營業收入淨額


In [67]:
營業收入淨額

symbol,000116,000930,000960,0009A0,1101,1102,1103,1104,1108,1109,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
date,,,,,,,,,,,,,,,,,,,,,
2013-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-25,1077908.0,764053.0,2254537.0,2057030.0,29808661.0,20736080.0,489170.0,1294920.0,1131650.0,1957481.0,...,5578738.0,3646517.0,7433190.0,729077.0,695747.0,591410.0,5184336.0,270758.0,1177022.0,2037860.0
2020-11-26,1077908.0,764053.0,2254537.0,2057030.0,29808661.0,20736080.0,489170.0,1294920.0,1131650.0,1957481.0,...,5578738.0,3646517.0,7433190.0,729077.0,695747.0,591410.0,5184336.0,270758.0,1177022.0,2037860.0
2020-11-27,1077908.0,764053.0,2254537.0,2057030.0,29808661.0,20736080.0,489170.0,1294920.0,1131650.0,1957481.0,...,5578738.0,3646517.0,7433190.0,729077.0,695747.0,591410.0,5184336.0,270758.0,1177022.0,2037860.0


In [63]:
data.get('financial_statement:營業收入淨額')


symbol,000116,000930,000960,0009A0,1101,1102,1103,1104,1108,1109,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
date,,,,,,,,,,,,,,,,,,,,,
2013-Q1,NaN,NaN,NaN,NaN,24114047.0,13931550.0,741189.0,1248072.0,1203671.0,916242.0,...,3983950.0,2553355.0,3496126.0,561304.0,877797.0,904775.0,3539905.0,1467930.0,1206462.0,716529.0
2013-Q2,NaN,NaN,NaN,NaN,28670184.0,18181256.0,1051146.0,1217277.0,1121734.0,1010171.0,...,4765801.0,3408953.0,3590615.0,669987.0,803340.0,1047579.0,3669592.0,1403828.0,1195298.0,996568.0
2013-Q3,NaN,NaN,NaN,NaN,30106839.0,17609788.0,945725.0,1295292.0,1195803.0,985555.0,...,4802503.0,3040973.0,3456868.0,645770.0,849737.0,947240.0,3608194.0,2040661.0,1008562.0,917564.0
2013-Q4,NaN,NaN,NaN,NaN,33207877.0,20450187.0,1530245.0,1515516.0,1348382.0,1639272.0,...,3053366.0,3088274.0,3649403.0,659058.0,747396.0,1059029.0,4283937.0,578521.0,901488.0,842126.0
2014-Q1,NaN,NaN,NaN,NaN,26716227.0,16256080.0,1019429.0,1379161.0,1219347.0,1192024.0,...,4331202.0,2205176.0,3706153.0,710276.0,859905.0,1010776.0,2965491.0,479306.0,661912.0,637597.0
2014-Q2,NaN,NaN,NaN,NaN,31873459.0,21194832.0,890073.0,1456386.0,1276260.0,1493124.0,...,4849561.0,2452832.0,3796011.0,764382.0,779265.0,1155528.0,3411138.0,28310.0,639967.0,918387.0
2014-Q3,NaN,NaN,NaN,NaN,30094955.0,19783979.0,908502.0,1406779.0,1236683.0,944057.0,...,4797940.0,2022525.0,3967048.0,715471.0,811014.0,1013291.0,3002027.0,17101.0,657955.0,641712.0
2014-Q4,NaN,NaN,NaN,NaN,29641168.0,20448390.0,1119682.0,1577578.0,1366908.0,1370884.0,...,3247771.0,2218682.0,4168125.0,713737.0,783636.0,961575.0,2913244.0,116816.0,922523.0,690596.0
2015-Q1,NaN,NaN,NaN,NaN,21440142.0,15362530.0,671355.0,1327430.0,1290902.0,951789.0,...,4153767.0,1809631.0,4137741.0,697281.0,880080.0,931870.0,2165674.0,137577.0,759412.0,690518.0


In [61]:
市值營收比.describe()

/Users/max/Library/Python/3.11/lib/python/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning:

invalid value encountered in subtract



symbol,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
count,2793.000000,2795.000000,2795.000000,2796.000000,2795.000000,2771.000000,2787.000000,2795.000000,2790.000000,2795.000000,...,2796.000000,2796.000000,2795.000000,2796.000000,2786.000000,2795.000000,2781.000000,2796.000000,2796.000000,2796.000000
mean,7132.990948,6905.173456,21209.606563,11804.464887,3791.058526,3791.300051,23523.696898,2316.088619,4353.240367,1480.184153,...,3673.684360,7976.256899,7380.331167,10327.548292,11208.464914,3325.122555,18333.195208,21664.698678,2627.096013,12657.032040
std,1870.311721,1090.028683,8862.310006,1745.158481,757.438812,908.153126,4795.280811,598.950942,1065.863300,566.064203,...,815.056495,2181.131706,3569.568825,2649.950931,2592.685594,903.975456,5861.503716,43857.573016,988.302861,10941.910739
min,4150.528532,4607.245273,7444.481653,6831.016650,2370.583739,2278.874564,13824.828807,1383.858349,2820.300347,627.214194,...,1986.048538,2708.639255,4069.819657,5730.018714,5946.538376,1852.332439,6821.639577,1046.605349,1411.742533,1859.858274
25%,5556.296112,6137.306680,12680.548660,10725.910095,3313.015427,3119.695318,19275.160901,1901.216591,3604.688376,913.148795,...,3126.057848,6065.928077,4970.545524,8591.772452,10060.083244,2562.357684,13959.777433,4934.128024,1915.936295,2834.911923
50%,6335.533292,6887.884258,21039.431288,11493.200343,3726.361557,3511.325929,23738.510919,2099.851160,3962.210823,1457.661309,...,3507.861671,8130.128588,5844.903823,9502.197763,11319.011892,3348.888560,17476.910198,12487.464863,2363.695262,11346.923636
75%,8570.671125,7472.447466,29222.356696,12972.070764,4113.009323,4396.838199,27778.241027,2609.227817,5183.439655,1978.397169,...,3946.344379,9549.722549,7793.205098,11520.444754,12978.653024,3901.887295,21531.552644,17524.221076,3016.220097,18357.949555
max,12155.602210,11896.544531,43160.391326,16426.486561,6705.993461,6243.278955,36751.549224,4508.677057,9143.556185,2679.638984,...,6474.068230,12816.068910,17589.679317,18738.036789,18987.807350,7233.223266,39561.523692,262853.347757,6366.364146,45798.269343


In [11]:
自由現金流.isnull().sum()

symbol
000116    811
000930    811
000960    811
0009A0    811
1101       42
         ... 
9944       41
9945       42
9946       42
9955       40
9958       41
Length: 1045, dtype: int64

In [24]:
股東權益報酬率.columns

Index(['000116', '000930', '000960', '0009A0', '1101', '1102', '1103', '1104',
       '1108', '1109',
       ...
       '9939', '9940', '9941', '9942', '9943', '9944', '9945', '9946', '9955',
       '9958'],
      dtype='object', name='symbol', length=1045)

In [ ]:
市值營收比 

In [56]:
condition2.sum().sum()

715010

In [60]:
市值營收比

symbol,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
date,,,,,,,,,,,,,,,,,,,,,
2013-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29,8600.283396,7089.335699,30775.012715,15440.768349,5650.917156,3180.694947,29338.816365,1801.899307,8923.985367,2520.239237,...,6448.275130,6051.953871,16220.471964,15341.582371,16288.119101,3726.625218,20710.366289,24603.910799,2139.535268,20205.551142
2024-01-02,8563.266394,7038.025670,30451.915469,15440.768349,5650.917156,3171.980714,29338.816365,1801.899307,9143.556185,2537.471642,...,6319.309627,6041.850442,16438.489061,15341.582371,16234.125336,3735.849538,21533.294751,25327.555234,2130.766681,20089.760018
2024-01-03,8439.876388,6952.508955,30209.592534,15336.790111,5633.034507,3154.552249,29104.730064,1801.899307,9033.770776,2533.163541,...,6345.102728,6001.436727,16307.678803,15170.486656,16234.125336,3726.625218,21231.554315,25206.947828,2130.766681,20552.924514


In [59]:
(市值營收比 < 8).sum().sum()

613

In [49]:
# 1608035 are selected 
condition1 = (市值 < 10000000000)
 # 715010 are selected 
condition2 = 自由現金流 > 0
#718631 are selected
condition3 = 股東權益報酬率 > 0
# 314006 are selected
condition4 = 營業利益成長率 > 0
# only 613 are selected
condition5 = 市值營收比 < 5

conditions =  condition2 & condition3 & condition4 #& condition5


conditions

symbol,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
date,,,,,,,,,,,,,,,,,,,,,
2013-04-19,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2013-04-22,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2013-04-23,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2013-04-24,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2013-04-25,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2024-01-02,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2024-01-03,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [50]:
conditions.sum().sum()

35

In [48]:
report =sim(position=conditions, resample='Q', trade_at_price='open',position_limit=1/5, fee_ratio=1.425/1000*0.3),# stop_loss=0.3655)
report.display_mae_mfe_analysis()

,annualized_rate_of_return,sharpe,max_drawdown,win_ratio
,12.55%,0.73,-29.65%,52.09%


,entry_date,exit_date,entry_sig_date,exit_sig_date,position,period,entry_index,exit_index,return,trade_price@entry_date,...,警示股@exit_date,處置股@exit_date,全額交割股@exit_date,mae,gmfe,bmfe,mdd,pdays,weight,next_weights
stock_id,,,,,,,,,,,,,,,,,,,,,
1101 台泥,2020-10-05,NaT,2020-09-30,NaT,0.002653,63.0,1899.0,-1.0,0.039518,41.50,...,False,False,False,-0.030120,0.057831,0.000000,-0.030120,37.0,0.002653,0.002392
1104 環泥,2020-04-01,NaT,2020-03-31,NaT,0.003509,189.0,1773.0,-1.0,0.573959,15.75,...,False,False,False,0.000000,0.614742,0.000000,-0.093555,188.0,0.002653,0.002392
1108 幸福,2020-07-01,NaT,2020-06-30,NaT,0.004032,129.0,1833.0,-1.0,0.323515,9.86,...,False,False,False,-0.046653,0.347949,0.034483,-0.126016,117.0,0.002653,0.002392
1210 大成,2019-07-01,NaT,2019-06-28,NaT,0.004000,372.0,1590.0,-1.0,0.432684,40.00,...,False,False,False,-0.094441,0.476523,0.232869,-0.265487,329.0,0.002653,0.002392
1215 卜蜂,2020-10-05,NaT,2020-09-30,NaT,0.002653,63.0,1899.0,-1.0,0.093889,66.50,...,False,False,False,0.000000,0.111278,0.000000,-0.032967,62.0,0.002653,0.002392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9937 全國,NaT,NaT,2021-03-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,0.000000,0.002392
9939 宏全,NaT,NaT,2021-03-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,0.000000,0.002392
9940 信義,NaT,NaT,2021-03-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,0.000000,0.002392


AttributeError: 'tuple' object has no attribute 'display_mae_mfe_analysis'